<a href="https://colab.research.google.com/github/wahid028/Sentiment-Analysis/blob/main/NLP_with_Pytorch_Lightning_v1.02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required Installation

In [1]:
#install kaggle
!pip install -q kaggle

#install pytorch-lightning, transformers and torchmetrics
! pip install -q pytorch-lightning transformers torchmetrics

#install nltk, spacy, beautifulsoup4 and regex
! pip install -q nltk spacy beautifulsoup4 regex

# #additional installation
# ! pip install Cython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.4 MB/s eta 0:00:00


Import Libraries

In [2]:
import pandas as pd
import numpy as np
import nltk
import spacy
import re
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics

nltk.download("all")
from bs4 import BeautifulSoup
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torchmetrics import Accuracy, Precision, Recall
from sklearn.metrics import precision_recall_fscore_support
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [3]:
#Additional import
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Resource Check

In [4]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Data Collection

In [5]:
#upload the kaggle.json file
from google.colab import files
files.upload()

#create a kaggle directory
!mkdir ~/.kaggle

#copy the kaggle.json to kaggle directory
!cp kaggle.json ~/.kaggle/

#permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

#data that need to be downloaded
!kaggle competitions download -c tweet-sentiment-extraction

#unzip the data
!unzip tweet-sentiment-extraction.zip

Saving kaggle.json to kaggle.json
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 127MB/s]
Archive:  tweet-sentiment-extraction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


Loading Datasets

In [6]:
#load the datasets
train = pd.read_csv('../content/train.csv')
test = pd.read_csv('../content/test.csv')

In [7]:
# drop the rows with neutral sentiment as we are only interested in positive and negative sentiment
train = train[train['sentiment'] != 'neutral']
train = train.reset_index(drop=True)

test = test[test['sentiment'] != 'neutral']
test = test.reset_index(drop=True)

#re-arranging the datasets
train = train[['text','sentiment']]
test = test[['text','sentiment']]

#sentiment converter funtion
def sentiment_ts(sentiment):
    if sentiment == 'negative':
        return 0
    elif sentiment == 'positive':
        return 1

#convert the text sentimnet into number
train['label'] = train['sentiment'].apply(sentiment_ts)
test['label'] = test['sentiment'].apply(sentiment_ts)

#final dataset
train = train[['text','label']]
test = test[['text','label']]

# #sanity check
# train.head()
# test.head()
# train.isnull().sum()
# test.isnull().sum()

Data Pre-Processing

In [ ]:
#load the spacy model and functions related to stop words modification
spacy_model = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords
# print(spacy_model.Defaults.stop_words) #print the default stop words list
# spacy_model.Defaults.stop_words.remove("whatever") #remove single stop word from the list
# spacy_model.Defaults.stop_words -= {"whatever", "whenever"} #remove several stop words from the list
# spacy_model.Defaults.stop_words.add("my_new_stopword") #add single stop word in the default list
# spacy_model.stop_words |= {"my_new_stopword1","my_new_stopword2"} #add several stop words in the default list

In [ ]:
#most widely used shortforms are in the internet for social media data
contra_Expan_Dict = {
"ain`t": "am not","aren`t": "are not","can`t": "cannot","can`t`ve": "cannot have","`cause": "because",
"could`ve": "could have","couldn`t": "could not","couldn`t`ve": "could not have","didn`t": "did not",
"doesn`t": "does not","don`t": "do not","hadn`t": "had not","hadn`t`ve": "had not have","hasn`t": "has not",
"haven`t": "have not","he`d": "he would","he`d`ve": "he would have","he`ll": "he will","he`ll`ve": "he will have",
"he`s": "he is","how`d": "how did","how`d`y": "how do you","how`ll": "how will",
"how`s": "how does","i`d": "i would","i`d`ve": "i would have","i`ll": "i will","i`ll`ve": "i will have","i`m": "i am",
"i`ve": "i have","isn`t": "is not","it`d": "it would","it`d`ve": "it would have","it`ll": "it will","it`ll`ve": "it will have",
"it`s": "it is","let`s": "let us","ma`am": "madam","mayn`t": "may not","might`ve": "might have","mightn`t": "might not",
"mightn`t`ve": "might not have","must`ve": "must have","mustn`t": "must not","mustn`t`ve": "must not have","needn`t": "need not","needn`t`ve": "need not have",
"o`clock": "of the clock","oughtn`t": "ought not","oughtn`t`ve": "ought not have","shan`t": "shall not",
"sha`n`t": "shall not","shan`t`ve": "shall not have","she`d": "she would",
"she`d`ve": "she would have","she`ll": "she will","she`ll`ve": "she will have",
"she`s": "she is","should`ve": "should have","shouldn`t": "should not","shouldn`t`ve": "should not have","so`ve": "so have","so`s": "so is",
"that`d": "that would","that`d`ve": "that would have","that`s": "that is","there`d": "there would","there`d`ve": "there would have","there`s": "there is",
"they`d": "they would","they`d`ve": "they would have","they`ll": "they will","they`ll`ve": "they will have","they`re": "they are","they`ve": "they have",
"to`ve": "to have","wasn`t": "was not"," u ": " you "," ur ": " your "," n ": " and ","won`t": "would not",
"dis": "this","bak": "back","brng": "bring","sooo": "so", "afaik" :"as far as i know",
"afk" :"away from keyboard","asap" :"as soon as possible","atk" :"at the keyboard","atm" :"at the moment","a3" :"anytime, anywhere, anyplace",
"bak" :"back at keyboard","bbl" :"be back later","bbs" :"be back soon","bfn" :"bye for now","b4n" :"bye for now","brb" :"be right back",
"brt" :"be right there","btw" :"by the way","b4" :"before","b4n" :"bye for now","cu" :"see you","cul8r" :"see you later",
"cya" :"see you","faq" :"frequently asked questions","fc" :"fingers crossed","fwiw" :"for what it's worth","fyi" :"for your information",
"gal" :"get a life","gg" :"good game","gn" :"good night","gmta" :"great minds think alike","gr8" :"great!","g9" :"genius",
"ic" :"i see","icq" :"i seek you (also a chat program)","ilu" :"ilu: i love you","imho" :"in my honest/humble opinion","imo" :"in my opinion","iow" :"in other words",
"irl" :"in real life","kiss" :"keep it simple, stupid","ldr" :"long distance relationship","lmao" :"laugh my a.. off","lol" :"laughing out loud",
"ltns" :"long time no see","l8r" :"later","mte" :"my thoughts exactly","m8" :"mate","nrn" :"no reply necessary","oic" :"oh i see",
"pita" :"pain in the a..","prt" :"party","prw" :"parents are watching","qpsa?   que pasa?" :"","rofl" :"rolling on the floor laughing",
"roflol" :"rolling on the floor laughing out loud","rotflmao" :"rolling on the floor laughing my ass off","sk8" :"skate","stats" :"your sex and age",
"asl" :"age, sex, location","thx" :"thank you","ttfn" :"ta-ta for now!","ttyl" :"talk to you later",
"u" :"you","u2" :"you too","u4e" :"yours for ever","wb" :"welcome back","wtf" :"what the fuck","wtg" :"way to go!",
"wuf" :"where are you from?","w8" :"wait","7k" :"sick:-d laugher", "w/out": "without", "ihavent": "i have not"}

In [ ]:
#function for converting shortforms to it's expanded form based on contra_Expan_Dict
def expanded_form(x):
  if x in contra_Expan_Dict.keys():
    return(contra_Expan_Dict[x])
  else:
    return(x)

In [ ]:
#function for removing url punctuations and digits
def clean_with_re(x):
  x=str(x)
  x=re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ", x) #Remove URLs
  x=re.sub(r'[^\w ]+', "", x) # Remove Punctuation-1
  x=re.sub(r"[,!@&\'?\.$%_]"," ", x) # Remove Punctuation-2
  x=re.sub(r"\d+"," ", x) #Remove digits
  return(x)

In [ ]:
#function for removing HTML Tags
def remove_html(text):
    return BeautifulSoup(text, "lxml").text

In [ ]:
#function for counting the words in the train dataset
from collections import Counter
cnt = Counter()
for text in train["text"].values:
    for word in text.split():
        cnt[word] += 1

#function for removing the most frequent words
cnt.most_common(10)
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

#function for removing the most rare words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [ ]:
#remove emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
#function for removing duplicate white spaces
def remove_duplicate_ws(x):
  x=str(x)
  x=" ".join(re.split("\s+", x, flags=re.UNICODE))
  return(x)

In [ ]:
def pre_processing(input_data, text_col):
  #convert all the input texts into lower case.
  input_data["text_col_clean"]=input_data[text_col].apply(lambda x:str(x).lower())
  #convert all the shortform of the input texts to its expanded form.
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[expanded_form(t) for t in str(x).split()])
  #remove the stopwords based on spacy default package
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:[t for t in x if t not in spacy_stopwords])
  #remove the url, punctuations and digits from the input text
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:clean_with_re(x))
  #remove the HTML Tags from the input text
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_html(x))
  #lemmatization - converting evary word to it's root form
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:" ".join([t.lemma_ for t in spacy_model(str(x))if t.lemma_ !="-PRON-" ]))
  #remove the most frequents words
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_freqwords(x))
  #remove the most rare words
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_rarewords(x))
  #remove the emojis
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_emoji(x))
  #remove the duplicate whitespace.
  input_data["text_col_clean"]=input_data["text_col_clean"].apply(lambda x:remove_duplicate_ws(x))

In [ ]:
#call the pre_processing function both for train and test datasets
pre_processing(input_data=train, text_col="text")
pre_processing(input_data=test, text_col="text")

In [ ]:
#check random text before and after pre-processing
print("Before Pre-processing: ",train["text"][300])
print("After Pre-processing: ",train["text_col_clean"][300])
#print the sentence length before and after pre-processing
print("Before Pre-processing: ",len(train["text"][300].split()))
print("After Pre-processing: ",len(train["text_col_clean"][300].split()))
#print the sentiment of the text
print("Sentiment of the text: ",train["label"][300])

Before Pre-processing:   but my bday is JUNE 19.. this is wack... and ihavent seen any promotions for my bday party  someone better finagle this asap!
After Pre-processing:  bday june wack have not see promotion bday party well finagle asap
Before Pre-processing:  23
After Pre-processing:  12
Sentiment of the text:  0


In [9]:
#Clean dataset
# train = train[['text_col_clean','label']]
# test = test[['text_col_clean','label']]

# #sanity check
# train.head()
# test.head()
# train.isnull().sum()
# test.isnull().sum()

#reduce the sample size inorder to save time for our experiment.
train = train.sample(n=1000)
test = test.sample(n=1000)

In [10]:
# Load train dataset and split into train and val dataset
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)

Define Parameters and Hyper parameters

In [11]:
# perparameters
learning_rate = 2e-5
max_length = 128
batch_size = 16
num_labels = 2
epochs = 1

pretrained_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
loss_fn = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Define custom class for the dataset

In [12]:
class MyDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text = self.data.iloc[index]['text'] # neec to change based on text pre-processing
        label = self.data.iloc[index]['label']
        encoding = self.tokenizer.encode_plus(
            text, 
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt')
        return encoding['input_ids'][0], encoding['attention_mask'][0], label

In [13]:
# Create datasets and data loaders for train val and test
train_dataset = MyDataset(train_data, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)

val_dataset = MyDataset(val_data, tokenizer, max_length)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=2)

test_dataset = MyDataset(test, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=2)

# #sanity check
# train_dataset[5]
# val_dataset[5]
# test_dataset[5]

Define Lightning Module class

In [14]:
class MyModel(pl.LightningModule):
    def __init__(self, num_labels):
        super().__init__()
        self.num_labels = num_labels
        self.model = pretrained_model
        self.loss_function = loss_fn
        
           
    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return output.logits
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, label = batch
        output = self(input_ids, attention_mask)
        loss = self.loss_function(output, label)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        preds = torch.argmax(output, dim=1)
        acc = (preds == label).float().mean()
        return {"loss": loss, "predictions": preds, "labels": label, "acc": acc}
    
    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, label = batch
        output = self(input_ids, attention_mask)
        loss = self.loss_function(output, label)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        preds = torch.argmax(output, dim=1)
                
        # calculate acc, precision, recall and f1 score
        acc = (preds == label).float().mean()
        preds = preds.cpu().numpy()
        targets = label.cpu().numpy()
        precision, recall, f1, _ = precision_recall_fscore_support(targets, preds, average='weighted')
        
        # log the metrics to the Lightning Logger
        self.log('val_acc', acc, logger=True)
        self.log('val_precision', precision, logger=True)
        self.log('val_recall', recall, logger=True)
        self.log('val_f1', f1, logger=True)

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, label = batch
        output = self(input_ids, attention_mask)
        loss = self.loss_function(output, label)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        preds = torch.argmax(output, dim=1)  

        # calculate acc, precision, recall and f1 score
        acc = (preds == label).float().mean()
        preds = preds.cpu().numpy()
        targets = label.cpu().numpy()
        precision, recall, f1, _ = precision_recall_fscore_support(targets, preds, average='weighted')
        
        # log the metrics to the Lightning Logger
        self.log('test_acc', acc, logger=True)
        self.log('test_precision', precision, logger=True)
        self.log('test_recall', recall, logger=True)
        self.log('test_f1', f1, logger=True) 
  
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

Model initialization checkpoin setup and Trainer setup

In [15]:
# Initialize the model
model = MyModel(num_labels)

# Load the best model checkpoint
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='./',
    filename='best_model'
)

# Custom loager. By default lightning_logs are generated. if you want to pass the custom logger then you have the pass logger=logger inside the trainer.
# logger = TensorBoardLogger("logs/", name="my_experiment")

# Early stoping
early_stop_callback = EarlyStopping(monitor="val_acc", min_delta=0.00, patience=3, verbose=False, mode="max")

trainer = pl.Trainer(max_epochs=epochs, gpus=torch.cuda.device_count(), 
                     callbacks=[checkpoint_callback, early_stop_callback, LearningRateMonitor()], profiler="simple", auto_lr_find=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Traing validation and testing along with result evaluation

In [16]:
# Training: This is the basic use of the trainer
trainer.fit(model, train_loader, val_loader)

# Validation: Perform an evaluation epoch over the validation set, outside of the training loop
trainer.validate(model, val_loader)

# Load the best model checkpoint for testing
best_model = MyModel.load_from_checkpoint(checkpoint_callback.best_model_path, num_labels=num_labels)

# Test the model using the test loader
trainer.test(best_model, test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                          | Params
----------------------------------------------------------------
0 | model         | BertForSequenceClassification | 109 M 
1 | loss_function | CrossEntropyLoss              | 0     
----------------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.935   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:pytorch_lightning.profilers.profiler:FIT Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                        

...

INFO:pytorch_lightning.profilers.profiler:VALIDATE Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                             

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8931451439857483
         test_f1            0.8930467894262109
        test_loss           0.3095775544643402
     test_precision         0.9079212789898273
       test_recall          0.8931451612903226
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


INFO:pytorch_lightning.profilers.profiler:TEST Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                 

[{'test_loss': 0.3095775544643402,
  'test_acc': 0.8931451439857483,
  'test_precision': 0.9079212789898273,
  'test_recall': 0.8931451612903226,
  'test_f1': 0.8930467894262109}]

In [17]:
#check the performance using tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

<IPython.core.display.Javascript object>